In [2]:
# Necessary Libraries
import numpy as np
import os
import pathlib
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from tensorflow import keras
import tensorflow as tf

In [4]:
# Define Constants
IMAGE_SHAPE = (200, 200)
DATA_DIR = 'C:/Users/Indhu/Downloads/bark_30/'  # Update with your texture dataset path
BATCH_SIZE = 100

In [6]:
# Load Data
data_dir = pathlib.Path(DATA_DIR)
classes = os.listdir(DATA_DIR)
texture_images_dict = {}
texture_labels_dict = {}


In [8]:

# Organize Data by Category
for index, category in enumerate(classes):
    texture_images_dict[category] = list(data_dir.glob(category + '/*'))
    texture_labels_dict[category] = index

X, y = [], []
for class_name, images in texture_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img, IMAGE_SHAPE)
        X.append(resized_img)
        y.append(texture_labels_dict[class_name])

X = np.array(X)
y = np.array(y)

In [10]:
# Normalize the Images
X = X / 255.0


In [12]:
# Split Dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [14]:

# Build Feature Extraction Model (Pretrained CNN)
base_model = keras.applications.MobileNetV2(input_shape=IMAGE_SHAPE + (3,), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the convolutional base

C:\Users\Indhu\AppData\Local\Temp\ipykernel_10368\3320221462.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = keras.applications.MobileNetV2(input_shape=IMAGE_SHAPE + (3,), include_top=False, weights='imagenet')


In [16]:
# Create the Feature Extraction Pipeline
feature_extractor = keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D()
])


In [18]:

# Extract Features
X_train_features = feature_extractor.predict(X_train, batch_size=BATCH_SIZE, verbose=1)
X_test_features = feature_extractor.predict(X_test, batch_size=BATCH_SIZE, verbose=1)

28/28 ━━━━━━━━━━━━━━━━━━━━ 107s 2s/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step


In [20]:
# Define Classifiers
classifiers = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=200, random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5),
    "Support Vector Machine": SVC(kernel='rbf', probability=True, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Naive Bayes": GaussianNB()
}


In [ ]:
# Train and Evaluate Classifiers
for model_name, model in classifiers.items():
    model.fit(X_train_features, y_train)
    y_pred = model.predict(X_test_features)
    print(f"{model_name} Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(classification_report(y_test, y_pred))


Random Forest Accuracy: 0.9075
              precision    recall  f1-score   support

           0       0.91      0.91      0.91        22
           1       0.95      0.86      0.90        21
           2       1.00      1.00      1.00        42
           3       0.93      1.00      0.96        52
           4       0.90      0.95      0.93        20
           5       0.91      1.00      0.95        20
           6       0.88      1.00      0.94        30
           7       1.00      1.00      1.00        21
           8       0.90      1.00      0.95        19
           9       0.91      1.00      0.96        32
          10       0.81      0.94      0.87        18
          11       0.97      0.93      0.95        30
          12       1.00      1.00      1.00        22
          13       1.00      1.00      1.00        15
          14       0.88      0.67      0.76        21
          15       1.00      1.00      1.00        15
          16       0.91      0.59      0.71       

C:\Users\Indhu\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\Indhu\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\Indhu\anaconda3\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Indhu\anaconda3\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\Indhu\anaconda3\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreatePro

K-Nearest Neighbors Accuracy: 0.9176
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        22
           1       0.87      0.95      0.91        21
           2       1.00      1.00      1.00        42
           3       0.96      1.00      0.98        52
           4       1.00      0.95      0.97        20
           5       0.95      1.00      0.98        20
           6       0.91      1.00      0.95        30
           7       1.00      1.00      1.00        21
           8       0.95      0.95      0.95        19
           9       1.00      0.97      0.98        32
          10       0.85      0.94      0.89        18
          11       0.97      0.97      0.97        30
          12       1.00      1.00      1.00        22
          13       1.00      1.00      1.00        15
          14       1.00      0.62      0.76        21
          15       1.00      1.00      1.00        15
          16       0.62      0.88      0.73 